In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
import tensorflow as tf

In [5]:
# Load data
file_path = 'C:/Users/WIN10/OneDrive/Documents/Bangkit/Dataset/inflasi_v3.csv'
df = pd.read_csv(file_path)


In [6]:
# Select relevant columns
df = df[['City', 'Month', 'Year', 'Inflation', 'CPI']]
print(df)

                        City     Month  Year  Inflation     CPI
0              KOTA MEULABOH   januari  2008       0.00    0.00
1            KOTA BANDA ACEH   januari  2008       2.26  204.80
2           KOTA LHOKSEUMAWE   januari  2008       1.89  160.79
3               KOTA SIBOLGA   januari  2008       2.51  163.31
4      KOTA PEMATANG SIANTAR   januari  2008       1.12  158.34
...                      ...       ...   ...        ...     ...
15657           KOTA TERNATE  november  2023      -0.20  115.29
15658              MANOKWARI  november  2023      -0.41  119.54
15659            KOTA SORONG  november  2023      -0.49  114.12
15660                MERAUKE  november  2023       0.30  115.32
15661          KOTA JAYAPURA  november  2023       0.10  112.99

[15662 rows x 5 columns]


In [7]:
# Remove 'KOTA' from the 'City' column
df['City'] = df['City'].str.replace('KOTA ', '', regex=False)

In [8]:
# Check if 'Inflation' and 'CPI' columns already contain numeric values
for col in ['Inflation', 'CPI']:
    if not pd.api.types.is_numeric_dtype(df[col]):
        # Replace commas with dots and convert to float
        df[col] = df[col].str.replace(',', '.').astype(float)

In [9]:
# Sort the data
df = df.sort_values(by=['City', 'Year', 'Month'])

In [10]:
# Scale inflation data and other numerical features
scaler = MinMaxScaler()
df[['Inflation', 'CPI']] = scaler.fit_transform(df[['Inflation', 'CPI']].values)

In [11]:
# Separate data into time series for each city
city_data = {}
for city in df['City'].unique():
    city_data[city] = df[df['City'] == city][['Inflation', 'CPI']].values

In [12]:
# Combine data for all cities into a single time series
all_cities_data = np.concatenate(list(city_data.values()))

# Function to create time series sequences
def create_time_series(data, time_steps=1):
    X, y = [], []
    for i in range(len(data) - time_steps + 1):
        a = data[i:(i + time_steps), :]
        X.append(a)
        y.append(data[i + time_steps - 1, 0])  # Assuming 'Inflation' is in the first column
    return np.array(X), np.array(y)

In [13]:
# Hyperparameters
time_steps = 12
n_features = 2

In [14]:
# Create time series for all cities
X_all, y_all = create_time_series(all_cities_data, time_steps)
X_all = X_all.reshape((X_all.shape[0], time_steps, n_features))

# Create and train a single LSTM model for all cities
model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(time_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(X_all, y_all, epochs=50, verbose=0)

In [15]:
# Example prediction for a city
sample_city = 'YOGYAKARTA'
input_data = scaler.transform(city_data[sample_city][-time_steps:])
input_data = input_data.reshape((1, time_steps, n_features))
predicted_inflation = model.predict(input_data)

1/1 [==============================] - 0s 286ms/step


In [16]:
# Inverse transform the prediction to get the actual value
predicted_inflation_actual = scaler.inverse_transform(np.concatenate([predicted_inflation, np.zeros_like(predicted_inflation)], axis=1))
predicted_inflation_actual = predicted_inflation_actual[:, 0]
print(f'Predicted Inflation for {sample_city}: {predicted_inflation_actual[0]}')

Predicted Inflation for YOGYAKARTA: 0.3589950203895569


In [17]:
# Compare with the most recent actual inflation value
historical_inflation = city_data[sample_city]
last_actual_inflation = historical_inflation[-1, 0]  # Assuming 'Inflation' is in the first column

# Determine the trend
if predicted_inflation_actual[0] > last_actual_inflation:
    trend = 'up'
elif predicted_inflation_actual[0] < last_actual_inflation:
    trend = 'down'
else:
    trend = 'unchanged'

print(f'Trend for {sample_city} inflation: {trend}')

Trend for YOGYAKARTA inflation: down
